<a href="https://colab.research.google.com/github/Bhadra-Indranil/LLM-for-Mental-Health/blob/main/GPT2_QA_Mental_Health_UL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df1_2020 = pd.read_csv("20200325_counsel_chat_processed.csv")
df1_2020.head()

,Unnamed: 0,questionTitle,questionText,topic,answerText,clean_answerText,clean_questionTitle,clean_questionText
0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"If everyone thinks you're worthless, then mayb...","if everyone thinks you're worthless, then mayb...",can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
1,1,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"Hello, and thank you for your question and see...","hello, and thank you for your question and see...",can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
2,2,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,First thing I'd suggest is getting the sleep y...,first thing i'd suggest is getting the sleep y...,can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
3,3,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,Therapy is essential for those that are feelin...,therapy is essential for those that are feelin...,can i change my feeling of being worthless to ...,i'm going through some things with my feelings...
4,4,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,I first want to let you know that you are not ...,i first want to let you know that you are not ...,can i change my feeling of being worthless to ...,i'm going through some things with my feelings...


In [ ]:
df1_2020 = df1_2020.drop(columns=['questionTitle','questionText','topic','answerText','clean_questionTitle'])
df1_2020.head()

,Unnamed: 0,clean_answerText,clean_questionText
0,0,"if everyone thinks you're worthless, then mayb...",i'm going through some things with my feelings...
1,1,"hello, and thank you for your question and see...",i'm going through some things with my feelings...
2,2,first thing i'd suggest is getting the sleep y...,i'm going through some things with my feelings...
3,3,therapy is essential for those that are feelin...,i'm going through some things with my feelings...
4,4,i first want to let you know that you are not ...,i'm going through some things with my feelings...


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel,Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

In [ ]:
class SimpleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length)

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        # Create item dictionary with tensors for each key.
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        # Set labels equal to input_ids for language modeling.
        item["labels"] = item["input_ids"].clone()
        return item

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
df1_2020['prompt'] = "Question: " + df1_2020['clean_questionText'] + " Answer: " + df1_2020['clean_answerText']
texts = df1_2020['prompt'].tolist()

In [ ]:
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
dataset = SimpleDataset(texts, tokenizer, max_length=128)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_steps=500,
    do_train=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
# 5a8e6c4186e9b161ceeb13f7a4d5d8bfc3ea1026
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: indranil9053rkm (indranil123iitd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
50,3.448500
100,3.047400
150,2.918500
200,2.796400
250,2.848900
300,2.758500
350,2.667800
400,2.590100
450,2.617800
500,2.596900


TrainOutput(global_step=5330, training_loss=1.800049284027844, metrics={'train_runtime': 1301.9487, 'train_samples_per_second': 16.352, 'train_steps_per_second': 4.094, 'total_flos': 1390726840320000.0, 'train_loss': 1.800049284027844, 'epoch': 10.0})

In [ ]:
trainer.save_model("./fine_tuned_gpt2_in_mental_health_question")

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2_in_mental_health_question")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set pad token to EOS and update the model config
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Determine the device (GPU if available, else CPU) and move the model to it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define your test prompt (using the same format as during training)
prompts = [
    "Question: I've been feeling anxious about the future and it's affecting my daily routine. What are some effective coping strategies? Answer:",
    "Question: I've been experiencing persistent feelings of depression and hopelessness. What steps can I take to improve my mood and seek help? Answer:",
    "Question: I often feel overwhelmed by stress at work and in my personal life. What techniques can help me manage stress better? Answer:",
    "Question: I struggle with negative self-talk and low self-esteem. How can I build a more positive self-image? Answer:",
    "Question: I'm finding it hard to cope with sudden changes and uncertainty in my life. What practices can help me build resilience? Answer:",
    "Question: I feel isolated and disconnected from those around me. What can I do to create a stronger support network? Answer:",
    "Question: I have trouble sleeping due to anxiety and racing thoughts. What are some effective methods to improve my sleep quality? Answer:",
    "Question: I occasionally experience panic attacks that leave me feeling out of control. What strategies might help me prevent or manage these episodes? Answer:"
]

for prompt in prompts:
    # Tokenize the prompt and move the resulting tensor to the same device as the model
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)  # Ensure attention to all tokens

    # Generate text from the prompt
    sample_outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Ensure attention mask is passed
        max_length=500,  # Allow longer responses
        num_beams=5,  # Beam search for better coherence
        no_repeat_ngram_size=2,  # Reduce repetition
        temperature=0.7,  # Balanced creativity
        top_p=0.9,  # Prevent extreme randomness
        eos_token_id=tokenizer.eos_token_id,  # Ensure stopping
    )

    # Decode and print the generated text
    for i, sample_output in enumerate(sample_outputs):
        text = tokenizer.decode(sample_output, skip_special_tokens=True)
        print(f"Generated Output {i+1}:\n{text}\n")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I've been feeling anxious about the future and it's affecting my daily routine. What are some effective coping strategies? Answer: there are many effective ways to deal with anxiety. some of them are as simple as opening a conversation with your primary care physician and asking if you'd like to have a discussion about a particular issue or issue. others are more complex, such as asking for an appointment with a therapist and attending a support group. and finally, you can schedule a time to talk with someone to discuss a specific issue/need. i recommend looking up the topic on the website of your choice, social media, or by calling 800-273-8255. the best way to get an idea of what is going on is to google "anxiety" and "situation." once you've identified the specific issues that are causing anxiety, consider what may be contributing to the anxiety and how best to address them.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I've been experiencing persistent feelings of depression and hopelessness. What steps can I take to improve my mood and seek help? Answer: there are a few steps you can take right now.1. find a therapist who specializes in working with people who are experiencing depression. they can help you figure out what are the key issues that are affecting you and what can be done to change them.2. talk to your primary care physician about the type of medication you are taking. if you're taking cialis or metronidazole, you may want to consider seeing a doctor to rule out other side effects.3. research has shown that when people take anti-depressants, they tend to feel better and feel less depressed. this may be a good thing to look into.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Generated Output 1:
Question: I'm finding it hard to cope with sudden changes and uncertainty in my life. What practices can help me build resilience? Answer: there are lots of different types of coping strategies that you can use to build resilient coping skills. here are a few of my favorites:



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I feel isolated and disconnected from those around me. What can I do to create a stronger support network? Answer: the best way to build a strong network of friends and family would be to find a therapist who specializes in working with people who are experiencing a lot of isolation and disconnection in their lives. a good therapist can help you identify the areas of your life that are most closely connected to each other and work with you to develop a plan of action that will help each of you feel more at ease and at peace with who you are.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Output 1:
Question: I have trouble sleeping due to anxiety and racing thoughts. What are some effective methods to improve my sleep quality? Answer: sleep is one of those areas that many people struggle with, but i think it would be helpful to have a little bit of a discussion with your primary care physician about some of the things that may be contributing to your sleep problems. i would also want to know if there are any medications that can be used to help you sleep better, or if you are looking for a sleep modality that you can use as a way to wake you up and start the day off right. there could be a number of things going on here, and i will try to give you some general ideas as to what might be causing the insomnia and how to get to a place of relief.

Generated Output 1:
Question: I occasionally experience panic attacks that leave me feeling out of control. What strategies might help me prevent or manage these episodes? Answer: it sounds like you may be experiencing a